**Table of Contents**
<div id="toc"></div>

# IMPORTS

In [1]:
from ete3 import Tree
import collections
from collections import Counter
from Bio import AlignIO
from Bio import SeqIO
import numpy as np
import math
#import pandas as pd
#import matplotlib.pyplot as plt
from __future__ import division
from scipy.spatial import distance
#%matplotlib inline

# 1610 EARLY SAMPLES

In [13]:
fw = open('wout17/mar_2014_ob_seqs.fa','w')
with open("wout17/Makona_1610_genomes_2016-06-23.fa", "rU") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        if record.id.split('|')[-1].startswith('2014-03'):
            fw.write('>{}\n{}\n'.format(record.id, record.seq))
fw.close()

Edited in geneious. Took all March 2014 samples from 1610 dataset. Manually extracted only CDS and fixed frameshift position. Then combined with the spliced 39 reference data set and removed the 2017 sequence. Realigned using geneious too. Exported doc as 'EBOV_CDS_splice_38_with_early_1610.fshift.aln.fa'.

# RAXML TREE

In [87]:
fw=open('wout17/EBOV_CDS_splice_38_with_early_1610.fshift.underscores.aln.fa','w')
with open('wout17/EBOV_CDS_splice_38_with_early_1610.aln.fa','r') as f:
    for l in f:
        l=l.rstrip('\n')
        l=l.replace('|','_')
        #l=l.replace('?','N')
        fw.write(l+'\n')
fw.close()            

In [275]:
%%bash
cd wout17/
rm RAxML_*
raxmlHPC-PTHREADS -f a -x 12345 -p 12345 -\# 1000 -m GTRGAMMA -s EBOV_CDS_splice_38_with_early_1610.fshift.underscores.aln.fa -n raxml_rapid_spliced_cds_tree -o EBOV_AF272001_Mayinga_Yambuku_DRC_1976,EBOV_KC242801_deRoover_DRC_1976,EBOV_KC242791_Bonduni_Tandala_DRC_1977 -T 4


RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file

IMPORTANT WARNING: Alignment column 5104 contains only undetermined values which will be treated as missing data


IMPORTANT WARNING: Sequences EBOV_KM034556_G3677_SierraLeone_G_Kailahun_KissiTeng_2014-05-26 and EBOV_KM233089_G3814_SierraLeone_G_Kailahun_Jawie_2014-06-15 are exactly identical


IMPORTANT WARNING: Sequences EBOV_KJ660346_Kissidougou-C15_Kissidougou_Guinea_2014-03-17 and EBOV_Kissidougou-C15_KJ660346_GIN_Kissidougou_2014-03-17 are exactly identical


IMPORTANT WARNING: Sequences EBOV_KC242799_13709_Kikwit_DRC_1995 and EBOV_KC242796_13625_Kikwit_DRC_1995 are exactly identical


IMPORTANT WARNING: Sequences EBOV_KC242798_1Ikot_Gabon_1996 and EBOV_KC242794_2Nza_Gabon_1996 are exactly identical


IMPORTANT WARNING: Sequences EBOV_KC242793_1Eko_Gabon_1996 and EBOV_KC242797_1Oba_Gabon_1996 are exactly identical


IMPORTANT WARNING: Sequences EBOV_KC242793_1Eko_Gabon_1996 and EBOV_

# NT AND CODON DICTS


In [2]:
n_dict = {
        'B':['C','G','T'],
        'D':['A','G','T'],
        'H':['A','C','T'],
        'K':['G','T'],
        'M':['C','A'],
        'N':['A','C','G','T'],
        'R':['A','G'],
        'S':['C','G'],
        'V':['A','C','G'],
        'W':['A','T'],
        'Y':['C','T'],
        'W':['A','T'],
        '?':['?']
    }

reverse_n_dict = {}

for k in n_dict:
    reverse_n_dict[''.join(sorted(n_dict[k]))]=k

codontable = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }

no_codons_per_aa=collections.Counter()
for i in codontable:
    no_codons_per_aa[codontable[i]]+=1

# GENE LOCATION DICT

In [3]:
gene={}
for i in range(1,2221):
    gene[i]="NP"
for i in range(2221,3244):
    gene[i]="VP35"
for i in range(3244,4225):
    gene[i]="VP40"
for i in range(4225,6256):
    gene[i]="GP"
for i in range(6256,7123):
    gene[i]="VP30"
for i in range(7123,7879):
    gene[i]="VP24"
for i in range(7879,14518):
    gene[i]="L"
print(len(gene))
fw = open('gene_location.csv','w')
fw.write('Position,Gene\n')
for k in gene:
    fw.write(('{},{}\n').format(k,gene[k]))
fw.close()

14517


# FITCH ANCESTRAL RECONSTRUCTION

In [11]:
def deltran(my_tree):
    bases = []
    t2 = my_tree.copy()
    for node in t2.traverse("preorder"):
        if not node.is_root() and not node.is_leaf():
            if len(node.var)>1:
                dt = node.var.intersection(node.up.var)
                if dt:
                    node.var=dt
        if len(node.var)>1:
            node.var=set(reverse_n_dict[''.join(sorted(list(node.var)))])
        node.var = list(node.var)[0]
        bases.append((node.id, node.var))
#     print t2.get_ascii(attributes=["var","id"])  
    return bases

def acctran(my_tree):
    bases = []
    t3 = my_tree.copy()
    for node in t3.traverse("preorder"):
        if not node.is_root() and not node.is_leaf():
            if len(node.var)>1:
                at = node.var.difference(node.up.var)
                if at:
                    node.var=at
        
        if len(node.var)>1:
            node.var=set(reverse_n_dict[''.join(sorted(list(node.var)))])
        node.var = list(node.var)[0]
        bases.append((node.id, node.var))
#     print t3.get_ascii(attributes=["var","id"])  
    return bases


def fitch_algorithm(tree,nts,option):
    t = Tree(tree,format=1)
   
    changes_of_state = 0

    c=0
    for node in t.traverse("postorder"):
        c+=1
#         print node
        node.add_features(id=c)
        
        #add snp annotations to the tips of the tree
        if node.is_leaf(): 
            nt=nts[index_dict[node.name]]
            
            if nt in n_dict:
                node.add_features(var=set(n_dict[nt]))
            else:
                node.add_features(var=set(nt))

        if node.children:
            r = set(node.children[1].var)
            r = [n_dict[i] if i in n_dict else i for i in r] 
            r = set([x for sublist in r for x in sublist])
            
            l = set(node.children[0].var)
            l = [n_dict[i] if i in n_dict else i for i in l]
            l = set([x for sublist in l for x in sublist])        
            
            a = set.intersection(l, r)
            if len(set.intersection(l, r))==0:
                a = set.union(l,r)
                changes_of_state+=1
            node.var=a
#     print changes_of_state
#     print t.get_ascii(attributes=["var","id"])  
    if option == 'ACCTRAN':
        bases = acctran(t)
    elif option == 'DELTRAN':
        bases = deltran(t)
    else:
        print('ARGUMENT ISSUE')
    return bases
    

                
# t = Tree('((C,A),(C,(A,G)));', format=1)
# t = Tree('((C,A),(T,(C,C)));', format=1)
# t = Tree('((C,T),(A,(T,(A,T))));')


In [12]:
reconstruction_dict= {}

a = AlignIO.read("wout17/EBOV_CDS_splice_38_with_early_1610.fshift.underscores.aln.fa","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")


sequence_reconstruction= collections.defaultdict(str)
outer_id = 0
t2 = Tree("wout17/RAxML_bipartitions.raxml_rapid_spliced_cds_tree",format=1)
for node in t2.traverse("postorder"):
    outer_id +=1
    sequence_reconstruction[outer_id]=''
print outer_id
    
index_dict = {}
c=0

for record in a:
    index_dict[record.id.replace('|','_')]= c
    c +=1
    
base_position = 0
for i in range(len(a[0])): 
    base_position+=1
    bp = aa[:, i:i+1] 
    bp = [i[0] for i in bp]
#     print base_position, bp
    if len(set(bp))==1:
        for k in sequence_reconstruction:
            sequence_reconstruction[k]+=bp[0].rstrip('\n')
    else:
        bases = fitch_algorithm("wout17/RAxML_bipartitions.raxml_rapid_spliced_cds_tree",bp,"DELTRAN")
#         print bases
        for i in bases:
            sequence_reconstruction[i[0]]+=i[1]
            
c=0
with open("wout17/ancestral_reconstruction_DELTRAN_with_names.fasta","w") as fw:
#     fw.write('id,node_name\n')
    t = Tree("wout17/RAxML_bipartitions.raxml_rapid_spliced_cds_tree",format=1)
    for node in t.traverse("postorder"):
        c+=1
#         print node
        if node.is_leaf():
            fw.write(str('>')+node.name+'\n'+sequence_reconstruction[c]+'\n')
#             node.add_features(id=node.name)
        else:
            fw.write(str('>')+str(c)+'\n'+sequence_reconstruction[c]+'\n')
            node.name=c
            
print t.get_ascii(attributes=["name"])  

t.write(format=8, outfile="wout17/RAxML_bipartitions.raxml_rapid_spliced_cds_tree.node_labelled.nw")

103

            /-EBOV_AY354458_Zaire_Kikwit_DRC_1995
         /7|
        |  |   /-EBOV_JQ352763_Kikwit_Kikwit_DRC_1995-05-04
        |   \6|
        |     |   /-EBOV_KC242796_13625_Kikwit_DRC_1995
        |      \5|
        |         \-EBOV_KC242799_13709_Kikwit_DRC_1995
      /23
     |  |      /-EBOV_KP271020_Lomela-Lokolia19_DRC_2014-08-20
     |  |   /10
     |  |  |   \-EBOV_KP271018_Lomela-Lokolia16_DRC_2014-08-20
     |  |  |
     |  |  |      /-EBOV_KC242792_Gabon_Mekouka_Gabon_1994
     |   \22     |
     |     |   /17      /-EBOV_KC242795_1Mbie_Gabon_1996
     |     |  |  |   /14
     |     |  |   \16   \-EBOV_KC242797_1Oba_Gabon_1996
     |      \21     |
     |        |      \-EBOV_KC242793_1Eko_Gabon_1996
     |        |
     |        |   /-EBOV_KC242798_1Ikot_Gabon_1996
     |         \20
   /97            \-EBOV_KC242794_2Nza_Gabon_1996
  |  |
  |  |      /-EBOV_KC242800_Ilembe_Gabon_2002
  |  |   /26
  |  |  |   \-EBOV_KF113528_Kelle_1_DRC_2003
  |  |  |
  |  |  |   

# SNP CLASSIFICATION

In [4]:
# FUNCTION DEFINITIONS #

def check_and_count(tree, old_count, snp, major_variant, minor_variant, minor_freq):
    check = minor_major_monophyly_check(tree, snp, major_variant, minor_variant, minor_freq)
    return tuple(map(lambda x, y: x + y, old_count, check))

def minor_major_monophyly_check(tree, snp, major_variant, minor_variant, minor_freq):
#     print(snp, major_variant, minor_variant)
    count = (0,0,0,0)
    if minor_freq==1:
        snp_classifier['only_occurs_once'].append((snp, major_variant+ ':'+minor_variant))
        count = (1,0,0,0)
    else:
        minor_monophyly_check = tree.check_monophyly(values=[minor_variant], target_attr="snp") 
        if minor_monophyly_check[0]:
            snp_classifier['minor_monophyly'].append((snp, major_variant+ ':'+minor_variant))
            count = (0,1,0,0)
        elif not minor_monophyly_check[0]:
            major_monophyly_check = tree.check_monophyly(values=[major_variant], target_attr="snp")
            if major_monophyly_check[0]:
                snp_classifier['major_monophyly'].append((snp, minor_variant+ ':'+major_variant))
                count = (0,0,1,0)
            elif not major_monophyly_check[0]:
                snp_classifier['not_monophyly'].append((snp, major_variant + ':'+minor_variant))
                count = (0,0,0,1)
    return count

In [5]:
# CREATING THE SNP DICT #
snp_dict = {}

a = AlignIO.read("wout17/ancestral_reconstruction_DELTRAN_with_names.fasta","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")

for i in range(len(a[0])): #for each base up to the length of the alignment
    bp = aa[:, i:i+1] 
    bases = []
    for item in bp:
        bases.append(item[0])
    c =1
    if i > 5000: #18880 for full genome
        for ambig in ['N','?','-']:
            if ambig in bases:
                c +=1
        if len(set(bases))>c:
            snp_dict['snp_'+str(i+1)]=bases
    elif 'N' in bases:
        if len(set(bases))>2:
            snp_dict['snp_'+str(i+1)]=bases
    else:
        if len(set(bases))>1:
            snp_dict['snp_'+str(i+1)]=bases

print len(snp_dict)


895


In [6]:
# CREATING THE INDEX DICT #
# LETS YOU KNOW WHICH ITEM IN THE SNP DICT VALUE (I.E. BASES LIST) CORRESPONDS TO WHICH RECORD IN THE ALIGNMENT #
index_dict = {}
reverse_index_dict= {}
c=0
for record in a:
    index_dict[record.id]= c
    reverse_index_dict[c]=record.id
    c +=1


Manually add in root node name (this case 103) to the newick tree 'wout17/RAxML_bipartitions.raxml_rapid_spliced_cds_tree.node_labelled.nw'

In [7]:
# DEFINING COUNTERS #
simple_cases_counter = 0
complex_cases_counter = 0
early_gap =0
late_gap =0
ambiguous_base=0

SIMPLE = (0,0,0,0)
COMPLEX = (0,0,0,0)

# CLASSIFYING THE SNPS #
snp_classifier = collections.defaultdict(list)

for snp in sorted(snp_dict, key= lambda x: float(x.split('_')[1])): 
    snp_count = Counter()
    t = Tree("wout17/RAxML_bipartitions.raxml_rapid_spliced_cds_tree.node_labelled.nw",format=8)
    for node in t.traverse("preorder"):
        if node.is_leaf():
            node.add_features(snp=snp_dict[snp][index_dict[node.name]])  #add snp annotations to each tip
            snp_count[snp_dict[snp][index_dict[node.name]]]+=1 #count the leaf snp frequency
    #print t.get_ascii(attributes=["snp"])  
    snp_count = snp_count.most_common()   

    major_variant = snp_count[0][0]

    # SIMPLE CASE #
    if len(snp_count)==2:
        # e.g. snp_189 [('G', 68), ('A', 4)]
        simple_cases_counter +=1 

        minor_variant = snp_count[1][0]
        minor_freq = snp_count[1][1]

        if minor_variant != 'N' and minor_variant != '?':
            SIMPLE = check_and_count(t, SIMPLE, snp, major_variant, minor_variant, minor_freq)            
    else:
        # COMPLEX CASE #
        # eg. snp_18913 [('G', 68), ('N', 3), ('A', 2), ('R', 2), ('-', 2)] #
        complex_cases_counter +=1 

        for k in snp_count[1:]:
#             print(k)
            minor_variant = k[0]
            minor_freq = k[1]

            if k[0]=='N' or k[0]=='?':
                ambiguous_base += 1
            elif float(snp.split('_')[1]) > 5000:

                new_id_list = [i for i in index_dict.keys() if i not in ['EBOV_JQ352763_Kikwit_Kikwit_DRC_1995-05-04','EBOV_HQ613403_M-M_DRC_2007-08-31','EBOV_HQ613402_034-KS_DRC_2008-12-31']]    
                t.prune(new_id_list)
                COMPLEX = check_and_count(t, COMPLEX, snp, major_variant, minor_variant, minor_freq)  
            else:
                COMPLEX = check_and_count(t, COMPLEX, snp, major_variant, minor_variant, minor_freq) 



In [8]:
# RESULT PRINT OUT #
print '##### RESULTS #####'
print 'Simple, Complex, Total: ',simple_cases_counter, '+', complex_cases_counter,'=', simple_cases_counter+complex_cases_counter
print '\n'
print '##### SNP Classifier Dict #####'
for k in snp_classifier:
    key_dict = collections.defaultdict(list)
    for i in snp_classifier[k]:
        key_dict[i[0]].append(i[1])
    print k,'(All:', len(snp_classifier[k]), ', Unique:', len(key_dict), ')'
print '\n'    
print '##### Simple cases #####'
print '(only a single snp, minor monophyly, major monophyly, not monophyletic)'
print SIMPLE
print '\n'        
print '##### Not quite as simple cases #####'
print 'minor N or ? ', ambiguous_base
print 'early - ', early_gap
print 'late - ', late_gap
print '(only a single snp, minor monophyly, major monophyly, not monophyletic)'
print COMPLEX
print '\n'   
print '##### Tally of the two #####'
print '(', SIMPLE[0]+COMPLEX[0], SIMPLE[1]+COMPLEX[1], SIMPLE[2]+COMPLEX[2], SIMPLE[3]+COMPLEX[3], ')'

##### RESULTS #####
Simple, Complex, Total:  381 + 514 = 895


##### SNP Classifier Dict #####
major_monophyly (All: 230 , Unique: 230 )
not_monophyly (All: 40 , Unique: 40 )
only_occurs_once (All: 116 , Unique: 114 )
minor_monophyly (All: 522 , Unique: 520 )


##### Simple cases #####
(only a single snp, minor monophyly, major monophyly, not monophyletic)
(49, 260, 53, 19)


##### Not quite as simple cases #####
minor N or ?  507
early -  0
late -  0
(only a single snp, minor monophyly, major monophyly, not monophyletic)
(67, 262, 177, 21)


##### Tally of the two #####
( 116 522 230 40 )


# CLUSTER ASSIGNMENT

In [45]:
a = AlignIO.read("wout17/ancestral_reconstruction_DELTRAN_with_names.fasta","fasta")
fw = open('wout17/clustering_annotations.txt','w')
fw.write('taxa\tattrib1\n')
cluster_dict = {}
ob_dict = {}
c = 0
c2 = 0
for record in a:
    date = record.id.split('_')[-1]
    if len(record.id) > 4:
        c+=1
        date = record.id.split('_')[-1]
        if date in ['1976','1977']:
            cluster_dict[record.id]='A'
            ob_dict[record.id]='1976/1977 DRC'
            fw.write(record.id + '\tA\n')
            
        elif date.count('2014')==1 and record.id.count('Lomela')==1:
            cluster_dict[record.id]='B3'
            ob_dict[record.id]='2014 DRC'
            fw.write(record.id + '\tB3\n')
            
        elif date in ['1996','1994']:
            cluster_dict[record.id]='B1'
            ob_dict[record.id]='1994/1996 DRC'
            fw.write(record.id + '\tB1\n')
            
        elif date in ['1995','1995-05-04']:
            ob_dict[record.id]='1995 DRC'
            cluster_dict[record.id]='B2'
            fw.write(record.id + '\tB2\n')
            
        elif date.count('2014')==1:
            cluster_dict[record.id]='C'
            ob_dict[record.id]='2014-2016 WAf'
            fw.write(record.id + '\tC\n')
            
        elif date.count('2007')==1 or date.count('2008')==1:
            cluster_dict[record.id]='D'
            ob_dict[record.id]='2007 DRC'
            fw.write(record.id + '\tD\n')
            
        elif date in ['2002','2003']:
            cluster_dict[record.id]='E'
            ob_dict[record.id]='2002/2003 DRC'
            fw.write(record.id + '\tE\n')
            
#         elif date=='2017-05':
#             cluster_dict[record.id]='M'
#             fw.write(record.id + '\tM\n')
    else:
        c2 +=1
        if int(record.id) in range(101,103):
            cluster_dict[record.id]='A'
            fw.write(record.id + '\tA\n')
        
        elif record.id=='10':
            cluster_dict[record.id]='B3'
            fw.write(record.id + '\tB3\n')
        elif int(record.id) in range(14,22):
            cluster_dict[record.id]='B1'
            fw.write(record.id + '\tB1\n')
        elif int(record.id) in range(5,8):
            cluster_dict[record.id]='B2'
            fw.write(record.id + '\tB2\n')

        elif int(record.id) in range(46,95):
            cluster_dict[record.id]='C'
            fw.write(record.id + '\tC\n')
            
        elif int(record.id) in range(30,44):
            cluster_dict[record.id]='D'
            fw.write(record.id + '\tD\n')

        elif record.id=='26':
            cluster_dict[record.id]='E'
            fw.write(record.id + '\tE\n')
            
print (len(cluster_dict))


# for k in cluster_dict:
#     if cluster_dict[k] == 'D' or cluster_dict[k]=='E':
#         cluster_dict[k]='C'
# for k in cluster_dict:
#     print k, cluster_dict[k]

with open('wout17/clustering_annotations_just_the_tips.txt','w') as fw:
    fw.write('Taxa\tCluster\tDate\tYear\tOutbreak\n')
    for k in cluster_dict:
        if k.startswith('E'):
            date = k.split('_')[-1]
            if len(date)==4:
                year=date
                date = date + '-01-01'
            else:
                year = date.split('-')[0]
                
            if cluster_dict[k].startswith('B'):
                fw.write("{}\t{}\t{}\t{}\t{}\n".format(k,'B',date,year,ob_dict[k]))
            else:
                fw.write("{}\t{}\t{}\t{}\t{}\n".format(k,cluster_dict[k],date,year,ob_dict[k]))
                


97


# BRANCH CLUSTER COMPARISONS

In [38]:
cluster_ancestral_node = {'E':'26',
                  'D':'43',
                  'C':'94',
                  'B3':'10',
                  'B1':'21',
                  'B2':'7',
                  'A':'102'}
branch_clusters={'E':'26:96',
                  'D':'43:95',
                  'C':'94:95',
                 'C:D':'95:96',
                 'C:D:E':'96:97',
                  'B3':'10:22',
                  'B1':'21:22',
                  'B2':'7:23',
                 'B1:B3':'22:23',
                 'B1:B2:B3':'23:97',
                  'A':'102:97'}
rev_branch_clusters = {}
for k in branch_clusters:
    rev_branch_clusters[branch_clusters[k]]=k
    

In [41]:
a = AlignIO.read("wout17/ancestral_reconstruction_DELTRAN_with_names.fasta","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")
c=0
my_cluster=collections.defaultdict(list)

branch_comparison_cluster=collections.defaultdict(list)

fw = open('wout17/branch_mutations_comparing_node_and_parent_node.csv','w')
fw.write('Clusters,SNP,Position\n')
for k in branch_clusters:
#     print k
    one = branch_clusters[k].split(':')[0]
    two = branch_clusters[k].split(':')[1]
    for record1 in a:
        for record2 in a:
            if record1.name == one and record2.name == two:
#                 print k, record1.name, record2.name
                for i in snp_classifier:
                    for j in snp_classifier[i]:
                        location = int(j[0].split('_')[1])-1
                        if record1.seq[location] != record2.seq[location]:
                            my_cluster[k].append(location)
                            #print one, two, j[1], record1.seq[location], record2.seq[location]
                            #print 'Clusters ', k, 'Base ', record1.seq[location], record2.seq[location], j
                            fw.write("{},{},{}\n".format(k,j[1],location+1))
                        
fw.close()    

fw = open('wout17/branch_mutations_counts.csv','w')
fw.write('Cluster1,Cluster2,Count\n')
fw2 = open('wout17/cluster_vs_cluster_mutations.csv','w')
fw2.write('Clusters,Position\n')
cluster_vs_cluster_dict=collections.Counter()
cluster_vs_cluster_location=collections.defaultdict(list)

for k in my_cluster:
    for j in my_cluster:
        for loc in my_cluster[k]:
            if loc in my_cluster[j]:
                cluster_vs_cluster_dict[(k,j)]+=1
                cluster_vs_cluster_location[(k,j)].append(loc)
            else:
                cluster_vs_cluster_dict[(k,j)]+=0
                
clusters = []                
for k in sorted(cluster_vs_cluster_dict):
    fw.write("{},{},{}\n".format(k[0],k[1],cluster_vs_cluster_dict[k]))
    for j in cluster_vs_cluster_location[k]:
        k = sorted(list(set(list(k))))
        
        sk = '|'.join(k)
        if sk == 'C|D':
            if j != 4212 and j!= 10979:
                fw2.write("{},{}\n".format(sk,j))
                clusters.append(sk)
        else:

            fw2.write("{},{}\n".format(sk,j))
            clusters.append(sk)
        
clusters = list(set(clusters))

print sorted(clusters)
fw.close()
fw2.close()


['A', 'A|C:D', 'A|D', 'A|E', 'B1', 'B1:B2:B3', 'B1:B2:B3|C', 'B1:B2:B3|D', 'B1:B3', 'B1|B1:B2:B3', 'B1|C', 'B1|D', 'B2', 'B2|C', 'B2|D', 'B2|E', 'B3', 'B3|C', 'B3|D', 'B3|E', 'C', 'C:D', 'C:D:E', 'C:D:E|D', 'C:D|D', 'C|C:D', 'C|C:D:E', 'C|E', 'D', 'E']


In [47]:
a = AlignIO.read("wout17/ancestral_reconstruction_DELTRAN_with_names.fasta","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")
c=0
my_cluster=collections.defaultdict(list)
cluster_ancestral_dict=collections.Counter()
fw = open('wout17/branch_mutations.csv','w')
fw.write('Cluster1,Cluster2,SNP,Position\n')
for k1 in cluster_ancestral_node:
    for k2 in cluster_ancestral_node:
#     print k
        for record1 in a:
            for record2 in a:
                if record1.name == cluster_ancestral_node[k1] and record2.name == cluster_ancestral_node[k2]:
                    print k, record1.name, record2.name

EBOV_KC242797_1Oba_Gabon_1996 102 102
EBOV_KC242797_1Oba_Gabon_1996 102 94
EBOV_KC242797_1Oba_Gabon_1996 102 26
EBOV_KC242797_1Oba_Gabon_1996 102 43
EBOV_KC242797_1Oba_Gabon_1996 102 21
EBOV_KC242797_1Oba_Gabon_1996 102 7
EBOV_KC242797_1Oba_Gabon_1996 102 10
EBOV_KC242797_1Oba_Gabon_1996 94 102
EBOV_KC242797_1Oba_Gabon_1996 94 94
EBOV_KC242797_1Oba_Gabon_1996 94 26
EBOV_KC242797_1Oba_Gabon_1996 94 43
EBOV_KC242797_1Oba_Gabon_1996 94 21
EBOV_KC242797_1Oba_Gabon_1996 94 7
EBOV_KC242797_1Oba_Gabon_1996 94 10
EBOV_KC242797_1Oba_Gabon_1996 26 102
EBOV_KC242797_1Oba_Gabon_1996 26 94
EBOV_KC242797_1Oba_Gabon_1996 26 26
EBOV_KC242797_1Oba_Gabon_1996 26 43
EBOV_KC242797_1Oba_Gabon_1996 26 21
EBOV_KC242797_1Oba_Gabon_1996 26 7
EBOV_KC242797_1Oba_Gabon_1996 26 10
EBOV_KC242797_1Oba_Gabon_1996 43 102
EBOV_KC242797_1Oba_Gabon_1996 43 94
EBOV_KC242797_1Oba_Gabon_1996 43 26
EBOV_KC242797_1Oba_Gabon_1996 43 43
EBOV_KC242797_1Oba_Gabon_1996 43 21
EBOV_KC242797_1Oba_Gabon_1996 43 7
EBOV_KC242797_1Oba_Ga

In [162]:
print list(j[1])[-1]

T


In [48]:
fw = open('wout17/clustering_info_EBOV.csv','w')
fw.write('Position,SNP,Cluster\n')
cluster_record = {}

for k in sorted(snp_dict, key=lambda x: int(x.split('_')[1])):
    cluster_list = []
    
    for i in reverse_index_dict:
        for j in reverse_index_dict:
            if reverse_index_dict[i] in cluster_dict and reverse_index_dict[j] in cluster_dict:
                if snp_dict[k][j] == snp_dict[k][i]:
                    cluster_list.append(cluster_dict[reverse_index_dict[i]])
                
#                     print cluster_dict[reverse_index_dict[i]]
    cluster_list = sorted(list(set(cluster_list)))
    print cluster_list
#     print sorted(cluster_list)
#     if len(cluster_list)>7 or cluster_list==[]:
#         pass
#     else:
#     if sorted(cluster_list) == ['D', 'E']:
#     print k, cluster_list
#     for j in list(set(cluster_list)):
#       print k.split('_')[1], snp_dict[k][index_dict['17EHV-0121_DRC_2017-05']], j
    j = ":".join(cluster_list)
    if j == '':
        cluster_record[k]='NA'
#     elif j == 'A:B:C:M':
#         cluster_record[k]='NA'
    else:
        cluster_record[k]=j
    for p in snp_classifier:
        for q in snp_classifier[p]:
            if q[0]==k:
                snp=q[1]
    fw.write(k.split('_')[1]+','+ snp+','+ j+'\n')


fw.close()
# for i in cluster_record:
#     print i, cluster_record[i]

['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', '

['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', '

['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', '

['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', 'C', 'D', 'E']
['A', 'B1', 'B2', 'B3', '

# SNP CLASSIFIER & CLUSTER OUTPUT


In [9]:
def sort_ambiguous_cases(my_ambiguous_codon):
    #Returns a list
    normal_codons = []
    for i in list(my_ambiguous_codon):
        if i not in ['A','T','G','C']:
            for j in n_dict[i]:
                new_codon= my_ambiguous_codon.replace(i,j)
                normal_codons.append(codontable[new_codon])
    if normal_codons == []:
        return codontable[my_ambiguous_codon]
    else:
        return normal_codons
    
x =sort_ambiguous_cases('GAK')
print x

['E', 'D']


In [42]:
fw=open('wout17/identify_and_classify_snps_results.csv','w')
fw.write('Type,SNP,Major,Minor,Position,Gene,Position_in_Codon,Mutation,Cluster\n')

CNS=''
with open('wout17/ancestral_reconstruction_DELTRAN_with_names_consensus_sequence.fasta','r') as f: #fdoesn't exist
    for l in f:
        l = l.rstrip('\n')
        if l.startswith('>'):
            pass
        else:
            CNS=l
c=0
snp_type_dict = {}
for k in snp_classifier:
    for i in sorted(snp_classifier[k], key= lambda x : int(x[0].split('_')[1])):
        location = int(i[0].split('_')[1])
        codon_number = math.ceil(location/3)
        position_in_codon = (location-1)%3
        index_no = int(codon_number*3)
        codon = ''.join(CNS[index_no-3:index_no]) 
#         print codon,position_in_codon, codon[position_in_codon]
        major = codon[:position_in_codon] + i[1].split(':')[0] + codon[position_in_codon + 1:]
        minor = codon[:position_in_codon] + i[1].split(':')[1] + codon[position_in_codon + 1:]
        if major in codontable and minor in codontable:
            if codontable[major]!=codontable[minor]:
                snp_type_dict[i[0]]= 'Nonsynonymous'
            elif codontable[major]==codontable[minor]:
                snp_type_dict[i[0]]= 'Synonymous'
            mut = snp_type_dict[i[0]]
            cluster = cluster_record[i[0]]
            fw.write('{},{},{},{},{},{},{},{},{}\n'.format(k, i[1], i[1].split(':')[0], i[1].split(':')[1], location, gene[location], location%3, mut, cluster))

        else:
            pass
#             snp_type_dict[i[0]]='Synonymous'
#             #It was only a single case that is very likely synonymous
#             print '**Ambiguous cases**'
#             print k, i, location, codon
#             new_major =  sort_ambiguous_cases(major)
#             new_minor =  sort_ambiguous_cases(minor)
#             print 'major', major, new_major
#             print 'minor', minor, new_minor
            
fw.close()


NameError: name 'cluster_record' is not defined

# ADAR EDITING?

In [37]:
a = AlignIO.read("wout17/ancestral_reconstruction_DELTRAN_with_names.fasta","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")

fw = open('wout17/t_to_c_mutations.csv','w')
fw.write('Sequence,Position\n')
fw2 = open('wout17/t_to_c_mutations_tips.csv','w')
fw2.write('Sequence,Position,Date,Cluster\n')
c=0

adar_signals=collections.defaultdict(list)

for seq in aa:
    locations_with_t_to_c=[]
    for i in snp_classifier:
        for j in sorted(snp_classifier[i], key= lambda x : int(x[0].split('_')[1])):
            if j[1] == 'T:C':
                location = int(j[0].split('_')[1])
                if seq[location-1] == 'C':
                    locations_with_t_to_c.append(location)
                    fw.write("{},{}\n".format(a[c].name,location))
                    if a[c].name.startswith('E'):
                        date = a[c].name.split('_')[-1]
                        if len(date)==4:
                            date= date+'-01-01'
                        fw2.write("{},{},{},{}\n".format(a[c].name,location,date,cluster_dict[a[c].name]))
    loc_list = sorted(locations_with_t_to_c)
    #print a[c].name, loc_list

    for k in range(0,len(loc_list)):
        quad= loc_list[k:k+4]
        if quad[-1]-quad[0]<=300:
            if len(quad) == 4:
#                 print quad, a[c].name
                adar_signals[str(quad)].append(a[c].name)
    c+=1
print len(adar_signals)
fw.close()
fw2.close()
fw = open('wout17/adar_signals.csv','w')
fw.write('Sequence,Position,Date\n')
fw2 = open('wout17/adar_signals_tips.csv','w')
fw2.write('Sequence,Position,Date,Signal,Cluster\n')

c = 0 
for i in adar_signals:
    #print '***', i, '***'
    #print set(adar_signals[i])
    c +=1
    for j in adar_signals[i]:
        for k in i.lstrip('[').rstrip(']').split(','):
#             print snp_type_dict['snp_'+str(k)]
            date = j.split('_')[-1]
            if len(date)==4:
                date= date+'-00-00'
            fw.write(j + ',' + k + ','+date+'\n')
            if j.startswith('E'):
                fw2.write("{},{},{},{},{}\n".format(j,k,date,c,cluster_dict[j]))
fw.close()
fw2.close()


NameError: name 'cluster_dict' is not defined

# BASE COMPOSITION ANALYSIS

In [ ]:
def check_if_synonymous_mutation():
    position_in_codon = (location-1)%3
        index_no = int(codon_number*3)
        codon = ''.join(CNS[index_no-3:index_no]) 
#         print codon,position_in_codon, codon[position_in_codon]
        major = codon[:position_in_codon] + i[1].split(':')[0] + codon[position_in_codon + 1:]
        minor = codon[:position_in_codon] + i[1].split(':')[1] + codon[position_in_codon + 1:]
        if major in codontable and minor in codontable:
            if codontable[major]!=codontable[minor]:
                snp_type_dict[i[0]]= 'Nonsynonymous'
            elif codontable[major]==codontable[minor]:
                snp_type_dict[i[0]]= 'Synonymous'

In [13]:
b = AlignIO.read("wout17/1610_with_consensus_cds+ig_*gapsNotRemoved.fasta","fasta")
bb = np.array([list(rec) for rec in b], np.character, order="F")

base_content=collections.defaultdict(dict)
c=0
length_dict={}
for seq in bb:
    if 'consensus' not in b[c].name.split('_'):
        length_dict[c]=len(seq)
        base_content[b[c].name]={"One":collections.Counter(),"Two":collections.Counter(),
                                 "Three":collections.Counter(),"Intergenic":collections.Counter()}       
        location = 0
        
        for base in seq:
            location+=1
            if base!='-' and base !='?':
                if location >= 14522:
                    base_content[b[c].name]["Intergenic"][base]+=1
                elif location%3==1:
                    base_content[b[c].name]["One"][base]+=1
                elif location%3==2:
                    base_content[b[c].name]["Two"][base]+=1
                elif location%3==0:
                    base_content[b[c].name]["Three"][base]+=1        
    c+=1


In [21]:
fw = open('wout17/1610_base_content.csv','w')
fw.write('Sequence,Position,Base,Count,TotalAtPos,Percentage,Country,Town,Date\n')        
c = 0
for k in base_content:
    date=k.split('|')[-1]
    N_count = 0
    info = b[c].name.split('|')
    for i in base_content[k]:    
        summed = 0
        for j in base_content[k][i]:
            
            if j!='N':
                summed = summed + base_content[k][i][j]
            elif j=='N':
                N_count = N_count + base_content[k][i][j]
                
        for j in base_content[k][i]: 
            if j in ["A","T","G","C"]:
                fw.write("{},{},{},{},{},{},{},{},{}\n".format(k, i, j, base_content[k][i][j],summed,(base_content[k][i][j]/summed)*100,info[3],info[4],date))
                
    fw.write("{},{},{},{},{},{},{},{},{}\n".format(k, "Three", 'N', N_count ,length_dict[c],(N_count/length_dict[c])*100,info[3],info[4],date))

    c+=1
    
fw.close()

In [36]:
a = AlignIO.read("spliced_cds_work/ancestral_reconstruction_DELTRAN_with_names.fasta","fasta")
aa = np.array([list(rec) for rec in a], np.character, order="F")

base_content=collections.defaultdict(dict)
c=0
for seq in aa:
    
    if a[c].name.startswith('E'):
#     if 'consensus' not in a[c].name.split('_'):
    
        base_content[a[c].name]={"One":collections.Counter(),
                                 "Two":collections.Counter(),
                                 "Three":collections.Counter()}
        location = 0
        for base in seq:
            location+=1
            if base!='N' and base!='-' and base !='?':
                if location%3==1:
                    base_content[a[c].name]["One"][base]+=1
                elif location%3==2:
                    base_content[a[c].name]["Two"][base]+=1
                elif location%3==0:
                    base_content[a[c].name]["Three"][base]+=1
                
    c+=1
fw = open('spliced_cds_work/39_ref_base_content.csv','w')
fw.write('Sequence,Position,Base,Count,TotalAtPos,Percentage,Date\n')        
c = 0
for k in base_content:
    date=k.split('_')[-1]
    if len(date)==4:
        date = date+'-01-01'
    for i in base_content[k]:
        
        summed = 0
        for j in base_content[k][i]:
            summed = summed + base_content[k][i][j]
            c+=1
            
        for j in base_content[k][i]:
            fw.write("{},{},{},{},{},{},{}\n".format(k, i, j, base_content[k][i][j],summed,
                                                     (base_content[k][i][j]/summed)*100,
                                                     date))

fw.close()

# CODON USAGE ANALYSIS

In [26]:
b = AlignIO.read("wout17/1610_with_consensus_cds+ig_*gapsNotRemoved.fasta","fasta")
bb = np.array([list(rec) for rec in b], np.character, order="F")
c=0
codon_usage_summary = {}
codon_freq_vector = {}
for k in bb:
    if 'consensus' not in b[c].name.split('_'):
    #print len(k)
        count_list = []
        amino_acid_counter = collections.Counter()
        total_aa_count=0
        codon_usage_counter = collections.Counter()
        for triplet in range(0,14522,3):
            codon = ''.join(k[triplet:triplet+3])
            if len(codon)==3:
                if codon in codontable:
                    total_aa_count+=1
                    amino_acid_counter[codontable[codon]]+=1
                    codon_usage_counter[codon]+=1
        usage_list = []
        freq_list = []
        for i in sorted(codon_usage_counter):
            aacid=codontable[i]
            observed_count = codon_usage_counter[i]
            aa_count = amino_acid_counter[aacid]
            no_codons = no_codons_per_aa[aacid]
            E = aa_count/no_codons
            RSCU = round(codon_usage_counter[i]/E, 4)
            freq = 1000*(observed_count/total_aa_count)
            usage_list.append(RSCU)
            count_list.append((i, codon_usage_counter[i], aa_count, RSCU,total_aa_count, freq, E ))
            freq_list.append(freq)
            
        codon_usage_summary[b[c].name]=count_list
        codon_freq_vector[b[c].name]=freq_list
    c+=1
    
    
fw = open('wout17/1610_codon_usage_summary.csv','w')
fw.write('Sequence,Codon,Count,AA.Count,RSCU,Date,Total.AA.Count,freq,E\n')
for k in codon_usage_summary:
    date=k.split('|')[-1]
    for j in codon_usage_summary[k]:
        
        fw.write("{},{},{},{},{},{},{},{},{}\n".format(k, j[0],j[1],j[2],j[3],date,j[4],j[5],j[6]))

fw.close()


In [104]:
fw = open('wout17/1610_codon_usage_summary_with_dates.csv','w')
fw.write('Sequence,Date,Codon,Frequency,Human,Bat,Snake,One,Two,Three\n')
with open('wout17/1610_codon_usage_summary.csv','r') as f:
     for l in f:
        l = l.rstrip('\n')
        name = l.split(',')[0]
        date = name.split('|')[-1]
        tokens =  l.split(',')
        codon = tokens[1]
        if codon in human_usage_dict:
            fw.write("{},{},{},{},{},{},{},{},{},{}\n".format(tokens[0],date, codon, tokens[-2], human_usage_dict[codon],bat_usage_dict[codon], snake_usage_dict[codon],codon[0],codon[1],codon[2]))
fw.close()

# CODON USAGE BIAS EUCLIDEAN DISTANCE

In [29]:
def process_codon_usage_table(filename):
    usage_dict={}
    with open(filename,'r') as f:
        for l in f:
            l =l.rstrip('\n')
            if l.startswith('Table') or l=='':
                pass
            else:
                tokens= l.split(')')
                tokens = [i.lstrip('\xc2\xa0') for i in tokens][:-1]
                for i in tokens:
                    tokens2 = i.split('\xc2\xa0')
                    tokens2 = [i.lstrip('(') for i in tokens2 if i != ''][:-1]
                    usage_dict[tokens2[0]]= float(tokens2[1])
    print len(usage_dict)
    return usage_dict      

human_usage_dict = process_codon_usage_table('wout17/homo_sapiens_9696_codon_usage.txt')
bat_usage_dict = process_codon_usage_table('wout17/chiroptera_9397_codon_usage.txt')
snake_usage_dict = process_codon_usage_table('wout17/serpentes_8570_codon_usage.txt')

index = {}
c=0
for i in sorted(human_usage_dict):
    index[c]=i
    c+=1

64
64
64


In [30]:
codon_freq_vector_no_stops = {}

for i in codon_freq_vector:
    new_freqs = []
    c = 0
    for j in codon_freq_vector[i]:
        if index[c] not in ['TAA','TGA','TAG']:
            new_freqs.append(j)
        else:
            new_freqs.append(1)
        c+=1
    codon_freq_vector_no_stops[i]= new_freqs


In [32]:
fw = open("wout17/euclidean_distance_wrt_other_sp.csv",'w')
fw.write("Sequence,Human,Bat,Snake,Date\n")

c = 0

human =[human_usage_dict[i] if i not in ['TGA','TAA','TAG'] else '1' for i in sorted(human_usage_dict) ]
human = tuple([float(i) for i in human])
bat = [bat_usage_dict[i] if i not in ['TGA','TAA','TAG'] else '1' for i in sorted(bat_usage_dict) ]
bat = tuple([float(i) for i in bat])
snake = [snake_usage_dict[i] if i not in ['TGA','TAA','TAG'] else '1' for i in sorted(snake_usage_dict) ]
snake = tuple([float(i) for i in snake])


for i in sorted(codon_freq_vector_no_stops, key = lambda x : map(int, x.split('|')[-1].split('-'))):
    c +=1
    tokens = i.split('|')
    date = tokens[-1]
    b = tuple(codon_freq_vector_no_stops[i])
    dsth = distance.euclidean(human, b)
    dstb = distance.euclidean(bat, b)
    dsts = distance.euclidean(snake, b)
    fw.write("{},{},{},{},{}\n".format(i,dsth,dstb,dsts,date))
        
fw.close()